#<font color='orange'>Симнете го податочното множество</font>

In [ ]:
!gdown 14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO

Downloading...
From: https://drive.google.com/uc?id=14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO
To: /content/crypto_data.csv
100% 90.0k/90.0k [00:00<00:00, 36.7MB/s]


#<font color='orange'>Направете претпроцесирање на податоците</font>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("./crypto_data.csv")
df.head()

,Date,Close (BNB),Close (BTC),Close (USDT),Close (ETH)
0,09-11-17,1.99077,7143.580078,1.00818,320.884003
1,10-11-17,1.79684,6618.140137,1.00601,299.252991
2,11-11-17,1.67047,6357.600098,1.00899,314.681000
3,12-11-17,1.51969,5950.069824,1.01247,307.907990
4,13-11-17,1.68662,6559.490234,1.00935,316.716003


In [ ]:
df.isnull().sum()

Date            0
Close (BNB)     0
Close (BTC)     0
Close (USDT)    0
Close (ETH)     0
dtype: int64

In [ ]:
def get_prev(df):
  matrix = []
  for i in range(2, len(df['Date'])-1):
    to_add=[df['Close (USDT)'][i], df['Close (ETH)'][i], df['Close (BTC)'][i],  df['Close (BNB)'][i]]
    for j in range(1,3):
      day=df['Close (USDT)'][i-j]
      to_add.append(day)
    matrix.append(to_add)
  return matrix

In [ ]:
clean_df = pd.DataFrame(get_prev(df))
clean_df.columns = ['USDT', 'ETH', 'BTC', 'BNB', 'USDT 1 day ago', 'USDT 2 days ago']
clean_df.head()

,USDT,ETH,BTC,BNB,USDT 1 day ago,USDT 2 days ago
0,1.00899,314.681000,6357.600098,1.67047,1.00601,1.00818
1,1.01247,307.907990,5950.069824,1.51969,1.00899,1.00601
2,1.00935,316.716003,6559.490234,1.68662,1.01247,1.00899
3,1.00683,337.631012,6635.750000,1.59258,1.00935,1.01247
4,1.00318,333.356995,7315.540039,1.53045,1.00683,1.00935


#<font color='orange'>Поделето го на тренирачко и тестирачко множество во сооднос 80:20</font>

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(clean_df[clean_df.columns[1:]],  clean_df['USDT'], test_size=0.2)
X_train.head()

,ETH,BTC,BNB,USDT 1 day ago,USDT 2 days ago
982,236.153168,9164.231445,17.484709,0.998352,0.998795
94,845.257996,8598.309570,8.986780,1.000090,0.999251
588,309.380310,10701.691410,37.799755,1.002799,1.000227
525,173.751266,5337.886230,25.007528,1.005918,1.004995
1611,3211.866943,42207.671880,418.590668,1.000372,1.000288


In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#<font color='orange'>Искористете соодветен Boosting модел за проблемот</font>

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(learning_rate=0.6,max_depth=15,alpha=5,n_estimators=35)
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

[16:31:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


#<font color='orange'>Тестирајте го моделот на тестирачкото множество со метриките за евалуација на дадениот проблем</font>

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

print(mean_absolute_error(Y_test, y_pred))
print(r2_score(Y_test, y_pred))

0.0022950267407444514
0.4407455812579806


Пробував ро разни вредности на хиперпарамтерите. Значително подобри резултати добив со зголемување на алфа и learning rate. Се добиваат солидни вредности за мае и r^2.